In [2]:
!pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [6]:
!pip3 install minio
!pip3 install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.2 MB/s eta 0:00:0000:0100:01


In [52]:
from datetime import datetime

In [4]:
!pip install clickhouse_driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.0 MB/s eta 0:00:00a 0:00:01


In [22]:



from datetime import datetime
from kafka import KafkaConsumer, TopicPartition

from minio import Minio
from minio.error import S3Error
import json
from io import BytesIO
import os

from kafka import KafkaConsumer
TOPIC = 'clients_kafka'
BOOTSTRAP = 'kafka:9092'
ACCESS_KEY = 'minioadmin'
SECRET_KEY = 'minioadmin'


file_path = 'KAFKA/topic=clients_kafka/year=2024/month=8/day=11/2024-08-11 01:34:25.json'
from minio import Minio
BUCKET_NAME = 'bronze-layer'
client = Minio("minio:9000",
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    secure = False
)

data = client.get_object(BUCKET_NAME, file_path)
content = json.load(data)
print(f"Conteúdo do arquivo {file_path}:")
print(content)



Conteúdo do arquivo KAFKA/topic=clients_kafka/year=2024/month=8/day=11/2024-08-11 01:34:25.json:
[{'nome': 'Daniel', 'sobrenome': 'Viana', 'idade': 52, 'email': 'ramoscecilia@example.com', 'data_cadastro': '2024-02-04'}, {'nome': 'Luiz Otávio', 'sobrenome': 'Nunes', 'idade': 34, 'email': 'theo80@example.net', 'data_cadastro': '2024-01-20'}, {'nome': 'Murilo', 'sobrenome': 'Barbosa', 'idade': 56, 'email': 'davicavalcanti@example.com', 'data_cadastro': '2024-04-30'}, {'nome': 'Vitor Hugo', 'sobrenome': 'Lopes', 'idade': 38, 'email': 'isabel02@example.net', 'data_cadastro': '2024-03-27'}, {'nome': 'Luiz Henrique', 'sobrenome': 'Fernandes', 'idade': 28, 'email': 'brenopeixoto@example.com', 'data_cadastro': '2024-04-08'}, {'nome': 'Rebeca', 'sobrenome': 'Novaes', 'idade': 32, 'email': 'enzo-gabriel24@example.net', 'data_cadastro': '2024-04-21'}, {'nome': 'Luna', 'sobrenome': 'Pereira', 'idade': 21, 'email': 'marcelosilva@example.net', 'data_cadastro': '2024-06-01'}, {'nome': 'Ana Clara', 's

In [5]:
!pip install polars

In [12]:
import polars as pl
from datetime import date
content
df_insert = pl.DataFrame(content)
df_insert = df_insert.with_columns(
    pl.col("data_cadastro").str.strptime(pl.Date, format="%Y-%m-%d").alias("data_cadastro")
).with_columns(pl.lit(date(2023, 12, 31)).alias("effective_date"))\
.with_columns(pl.lit(date(2999, 12, 31)).alias("end_date"))\
.with_columns(pl.lit(1).alias("is_current"))
# Exibir o DataFrame
print(df_insert)

shape: (245, 8)
┌─────────────┬───────────┬───────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ nome        ┆ sobrenome ┆ idade ┆ email      ┆ data_cadas ┆ effective_ ┆ end_date   ┆ is_current │
│ ---         ┆ ---       ┆ ---   ┆ ---        ┆ tro        ┆ date       ┆ ---        ┆ ---        │
│ str         ┆ str       ┆ i64   ┆ str        ┆ ---        ┆ ---        ┆ date       ┆ i32        │
│             ┆           ┆       ┆            ┆ date       ┆ date       ┆            ┆            │
╞═════════════╪═══════════╪═══════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ Daniel      ┆ Viana     ┆ 52    ┆ ramoscecil ┆ 2024-02-04 ┆ 2023-12-31 ┆ 2999-12-31 ┆ 1          │
│             ┆           ┆       ┆ ia@example ┆            ┆            ┆            ┆            │
│             ┆           ┆       ┆ .com       ┆            ┆            ┆            ┆            │
│ Luiz Otávio ┆ Nunes     ┆ 34    ┆ theo80@exa ┆ 2024-01-20 ┆ 2023-12-31 ┆ 

In [13]:
from clickhouse_driver import Client

# Conexão ao servidor ClickHouse
client_ch = Client(
    host='clickhouse',       # Endereço do servidor ClickHouse
    port=9000,              # Porta padrão para conexões TCP
    user='default',         # Usuário (padrão é 'default')
    password='',            # Senha (por padrão, vazio)
    database='default',     # Nome do banco de dados (por padrão, 'default')
)

In [14]:
client_ch.execute('''
CREATE OR REPLACE TABLE dim_clients (
    nome String,
    sobrenome String,
    idade Int32,
    email String,
    data_cadastro Date,
    effective_date Date,
    end_date Date,
    is_current UInt8
) ENGINE = MergeTree()
ORDER BY (nome, sobrenome, effective_date)
''')

[]

In [15]:
# Converta o DataFrame para um formato que pode ser enviado ao ClickHouse
data = df_insert.to_dicts()

# Inserir os dados no ClickHouse
client_ch.execute(
    'INSERT INTO usuarios (nome, sobrenome, idade, email, data_cadastro, effective_date, end_date, is_current) VALUES',
    [(row['nome'], row['sobrenome'], row['idade'], row['email'], row['data_cadastro'], row['effective_date'], row['end_date'], row['is_current']) for row in data]
)

245

In [17]:
# Definir os nomes das colunas conforme a tabela do ClickHouse
result = client_ch.execute(f"SELECT * FROM usuarios where is_current = 1")
columns = ['nome', 'sobrenome', 'idade', 'email', 'data_cadastro',  'effective_date', 'end_date', 'is_current']

# Criar o DataFrame Polars a partir dos resultados
df_table = pl.DataFrame(result, schema=columns)

/tmp/ipykernel_423/3037111310.py:5: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df_table = pl.DataFrame(result, schema=columns)


In [19]:
import polars as pl
from datetime import date

for i in range(5):
    content[i]['data_cadastro'] = '2025-02-04'
    if i == 3:
        content[i]['data_cadastro'] = '2023-02-04'
        
df_insert = pl.DataFrame(content[0:5])

df_insert = df_insert.with_columns(
    pl.col("data_cadastro").str.strptime(pl.Date, format="%Y-%m-%d").alias("data_cadastro")
).with_columns(pl.lit(date.today()).alias("effective_date")).with_columns(pl.lit(date(2999, 12, 31)).alias("end_date")).with_columns(pl.lit(1).alias("is_current"))
# Exibir o DataFrame
print(df_insert)

shape: (5, 8)
┌─────────────┬───────────┬───────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ nome        ┆ sobrenome ┆ idade ┆ email      ┆ data_cadas ┆ effective_ ┆ end_date   ┆ is_current │
│ ---         ┆ ---       ┆ ---   ┆ ---        ┆ tro        ┆ date       ┆ ---        ┆ ---        │
│ str         ┆ str       ┆ i64   ┆ str        ┆ ---        ┆ ---        ┆ date       ┆ i32        │
│             ┆           ┆       ┆            ┆ date       ┆ date       ┆            ┆            │
╞═════════════╪═══════════╪═══════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ Daniel      ┆ Viana     ┆ 52    ┆ ramoscecil ┆ 2025-02-04 ┆ 2024-08-11 ┆ 2999-12-31 ┆ 1          │
│             ┆           ┆       ┆ ia@example ┆            ┆            ┆            ┆            │
│             ┆           ┆       ┆ .com       ┆            ┆            ┆            ┆            │
│ Luiz Otávio ┆ Nunes     ┆ 34    ┆ theo80@exa ┆ 2025-02-04 ┆ 2024-08-11 ┆ 29

In [208]:
"""
Nesse caso, eu utilizo o email como chave primaria (poderia ser id, cpf, etc...), faço o join com a tabela para ver os emails repetidos
que estão sendo inseridos e filtro pela data de cadastro para garantir que os dados repetidos que estamos inserindo
sao o cadastro mais atualizado.

"""
df_joined = df_insert.join(
    df_table, 
    on="email", 
    how="inner"
).filter(
    pl.col("data_cadastro").cast(pl.Date) > pl.col("data_cadastro_right").cast(pl.Date)
)

# Selecionar as colunas e renomear conforme necessário
df_update_inactive = df_joined.select([
    pl.col("nome"),
    pl.col("sobrenome"),
    pl.col("idade"),
    pl.col("email"),
    pl.col("data_cadastro_right").alias("data_cadastro"),
    pl.col("effective_date_right").alias("effective_date"),
    pl.col("effective_date").alias("end_date"),
    pl.lit(0).alias("is_current")
])


In [209]:
df_update_inactive

nome,sobrenome,idade,email,data_cadastro,effective_date,end_date,is_current
str,str,i64,str,date,date,date,i32
"""Daniel""","""Viana""",52,"""ramoscecilia@example.com""",2024-02-04,2023-12-31,2024-08-11,0
"""Luiz Henrique""","""Fernandes""",28,"""brenopeixoto@example.com""",2024-04-08,2023-12-31,2024-08-11,0
"""Luiz Otávio""","""Nunes""",34,"""theo80@example.net""",2024-01-20,2023-12-31,2024-08-11,0
"""Murilo""","""Barbosa""",56,"""davicavalcanti@example.com""",2024-04-30,2023-12-31,2024-08-11,0


In [210]:
# Converta o DataFrame para um formato que pode ser enviado ao ClickHouse
for row in df_update_inactive.to_dicts():
    client_ch.execute(
        """
        ALTER TABLE usuarios
        UPDATE end_date = %(end_date)s,
               is_current = %(is_current)s
        WHERE email = %(email)s
        """,
        {'end_date': row['end_date'], 'is_current': row['is_current'], 'email': row['email']}
    )



In [211]:
 client_ch.execute("SELECT * from usuarios where is_current = 1")

[('Alexandre',
  'Rezende',
  21,
  'goncalvesluiz-otavio@example.net',
  datetime.date(2024, 3, 28),
  datetime.date(2023, 12, 31),
  datetime.date(1970, 1, 1),
  1),
 ('Alexia',
  'Gonçalves',
  45,
  'ribeiromaria-clara@example.com',
  datetime.date(2024, 3, 12),
  datetime.date(2023, 12, 31),
  datetime.date(1970, 1, 1),
  1),
 ('Alícia',
  'da Rocha',
  24,
  'pedro-henrique88@example.org',
  datetime.date(2024, 1, 19),
  datetime.date(2023, 12, 31),
  datetime.date(1970, 1, 1),
  1),
 ('Amanda',
  'Moraes',
  44,
  'tda-conceicao@example.net',
  datetime.date(2024, 2, 14),
  datetime.date(2023, 12, 31),
  datetime.date(1970, 1, 1),
  1),
 ('Ana',
  'da Cunha',
  36,
  'qrodrigues@example.org',
  datetime.date(2024, 1, 22),
  datetime.date(2023, 12, 31),
  datetime.date(1970, 1, 1),
  1),
 ('Ana',
  'da Luz',
  26,
  'cecilia51@example.org',
  datetime.date(2024, 4, 8),
  datetime.date(2023, 12, 31),
  datetime.date(1970, 1, 1),
  1),
 ('Ana Beatriz',
  'Nunes',
  58,
  'vda-luz@e

In [187]:

""" 
caso onde o cliente inserido ja esta na base mas tem a data de cadastro menor do
que a já cadastrada, nesse caso, o dado é inserido mas o dado ja persistido
é mantido como o dado corrente porque tem a data de cadastro maior

"""
df_joined = df_insert.join(
    df_table, 
    on="email", 
    how="inner"
).filter(
    pl.col("data_cadastro").cast(pl.Date) < pl.col("data_cadastro_right").cast(pl.Date)
)

# Selecionar as colunas e renomear conforme necessário
df_update_inactive_new = df_joined.select([
    pl.col("nome"),
    pl.col("sobrenome"),
    pl.col("idade"),
    pl.col("email"),
    pl.col("data_cadastro"),
    pl.col("effective_date"),
    pl.col("effective_date").alias("end_date"),
    pl.lit(0).alias("is_current")
])


In [188]:
df_update_inactive_new

nome,sobrenome,idade,email,data_cadastro,effective_date,end_date,is_current
str,str,i64,str,date,date,date,i32
"""Vitor Hugo""","""Lopes""",38,"""isabel02@example.net""",2023-02-04,2024-08-11,2024-08-11,0


In [194]:
df_insert = df_insert.join(
    df_update_inactive_new, 
    on="email", 
    how="left"
).select([
    pl.col("nome"),
    pl.col("sobrenome"),
    pl.col("idade"),
    pl.col("email"),
    pl.col("data_cadastro"),
    pl.when(pl.col("nome_right").is_not_null()).then(pl.col("effective_date_right")).otherwise(pl.col("effective_date")).alias("effective_date"),
    pl.when(pl.col("nome_right").is_not_null()).then(pl.col("end_date_right")).otherwise(pl.col("end_date")).alias("end_date"),
     pl.when(pl.col("nome_right").is_not_null()).then(pl.col("is_current_right")).otherwise(pl.col("is_current")).alias("is_current"),
])

In [197]:
df_insert

nome,sobrenome,idade,email,data_cadastro,effective_date,end_date,is_current
str,str,i64,str,date,date,date,i32
"""Daniel""","""Viana""",52,"""ramoscecilia@example.com""",2025-02-04,2024-08-11,2999-12-31,1
"""Luiz Otávio""","""Nunes""",34,"""theo80@example.net""",2025-02-04,2024-08-11,2999-12-31,1
"""Murilo""","""Barbosa""",56,"""davicavalcanti@example.com""",2025-02-04,2024-08-11,2999-12-31,1
"""Vitor Hugo""","""Lopes""",38,"""isabel02@example.net""",2023-02-04,2024-08-11,2024-08-11,0
"""Luiz Henrique""","""Fernandes""",28,"""brenopeixoto@example.com""",2025-02-04,2024-08-11,2999-12-31,1


In [195]:
# Converta o DataFrame para um formato que pode ser enviado ao ClickHouse
data = df_insert.to_dicts()

# Inserir os dados no ClickHouse
client_ch.execute(
    'INSERT INTO usuarios (nome, sobrenome, idade, email, data_cadastro, effective_date, end_date, is_current) VALUES',
    [(row['nome'], row['sobrenome'], row['idade'], row['email'], row['data_cadastro'], row['effective_date'], row['end_date'], row['is_current']) for row in data]
)

5

In [20]:
clientes_existentes = client_ch.execute("SELECT nome, sobrenome, idade, email FROM usuarios")
clientes_existentes_dict = {cliente[3]: {"nome": cliente[0], "sobrenome": cliente[1], "idade": cliente[2]} for cliente in clientes_existentes}

In [21]:
clientes_existentes_dict

{'goncalvesluiz-otavio@example.net': {'nome': 'Alexandre',
  'sobrenome': 'Rezende',
  'idade': 21},
 'ribeiromaria-clara@example.com': {'nome': 'Alexia',
  'sobrenome': 'Gonçalves',
  'idade': 45},
 'pedro-henrique88@example.org': {'nome': 'Alícia',
  'sobrenome': 'da Rocha',
  'idade': 24},
 'tda-conceicao@example.net': {'nome': 'Amanda',
  'sobrenome': 'Moraes',
  'idade': 44},
 'qrodrigues@example.org': {'nome': 'Ana',
  'sobrenome': 'da Cunha',
  'idade': 36},
 'cecilia51@example.org': {'nome': 'Ana', 'sobrenome': 'da Luz', 'idade': 26},
 'vda-luz@example.com': {'nome': 'Ana Beatriz',
  'sobrenome': 'Nunes',
  'idade': 58},
 'ada-cunha@example.net': {'nome': 'Ana Beatriz',
  'sobrenome': 'Ramos',
  'idade': 56},
 'vitor-hugoviana@example.net': {'nome': 'Ana Clara',
  'sobrenome': 'da Luz',
  'idade': 60},
 'isaac44@example.org': {'nome': 'Ana Clara',
  'sobrenome': 'da Mata',
  'idade': 51},
 'joana70@example.org': {'nome': 'Ana Julia',
  'sobrenome': 'Correia',
  'idade': 52},
 '

In [28]:
read_table().is_empty()

True

In [35]:
import polars as pl
from datetime import date

def read_table():
    client_ch.execute('''
    CREATE TABLE IF NOT EXISTS dim_clients (
        nome String,
        sobrenome String,
        idade Int32,
        email String,
        data_cadastro Date,
        effective_date Date,
        end_date Date,
        is_current UInt8
    ) ENGINE = MergeTree()
    ORDER BY (nome, sobrenome, effective_date)
    ''')
    result = client_ch.execute(f"SELECT * FROM dim_clients where is_current = 1")
    columns = ['nome', 'sobrenome', 'idade', 'email', 'data_cadastro',  'effective_date', 'end_date', 'is_current']
    # Criar o DataFrame Polars a partir dos resultados
    df_table = pl.DataFrame(result, schema=columns) 
    return df_table

def insert_dim_clients(content):
    #Criando dataframe polars com os dados a serem inseridos
    df_insert = pl.DataFrame(content)
    df_insert = df_insert.with_columns(
        pl.col("data_cadastro").str.strptime(pl.Date, format="%Y-%m-%d").alias("data_cadastro")
    ).with_columns(pl.lit(date(2023, 12, 31)).alias("effective_date"))\
    .with_columns(pl.lit(date(2149, 6, 6)).alias("end_date"))\
    .with_columns(pl.lit(1).alias("is_current"))
    
    #lendo a tabela para fazer o scd2
    df_table = read_table()
    if df_table.is_empty():
        data = df_insert.to_dicts()
        
        # Inserir os dados no ClickHouse
        client_ch.execute(
            'INSERT INTO dim_clients (nome, sobrenome, idade, email, data_cadastro, effective_date, end_date, is_current) VALUES',
            [(row['nome'], row['sobrenome'], row['idade'], row['email'], row['data_cadastro'], row['effective_date'], row['end_date'], row['is_current']) for row in data]
        )
        print("Dados inseridos com sucesso")
    else:  
        """
        Nesse caso, eu utilizo o email como chave primaria (poderia ser id, cpf, etc...), faço o join com a tabela para ver os emails repetidos
        que estão sendo inseridos e filtro pela data de cadastro para garantir que os dados repetidos que estamos inserindo
        sao o cadastro mais atualizado.
        
        """
        df_joined = df_insert.join(
            df_table, 
            on="email", 
            how="inner"
        ).filter(
            pl.col("data_cadastro").cast(pl.Date) > pl.col("data_cadastro_right").cast(pl.Date)
        )
        
        # Linhas a serem inativadas com is_current = 0 e fechamento do end date
        df_update_inactive = df_joined.select([
            pl.col("nome"),
            pl.col("sobrenome"),
            pl.col("idade"),
            pl.col("email"),
            pl.col("data_cadastro_right").alias("data_cadastro"),
            pl.col("effective_date_right").alias("effective_date"),
            pl.col("effective_date").alias("end_date"),
            pl.lit(0).alias("is_current")
        ])
        for row in df_update_inactive.to_dicts():
            client_ch.execute(
                """
                ALTER TABLE dim_clients
                UPDATE end_date = %(end_date)s,
                       is_current = %(is_current)s
                WHERE email = %(email)s
                """,
                {'end_date': row['end_date'], 'is_current': row['is_current'], 'email': row['email']}
            )
        
        """ 
        caso onde o cliente a ser inserido ja esta na base mas tem a data de cadastro menor do
        que a já cadastrada, nesse caso, o dado é inserido mas o dado ja persistido
        é mantido como o dado corrente porque tem a data de cadastro maior
        
        """
        df_joined = df_insert.join(
            df_table, 
            on="email", 
            how="inner"
        ).filter(
            pl.col("data_cadastro").cast(pl.Date) < pl.col("data_cadastro_right").cast(pl.Date)
        )
        # Selecionar as colunas e renomear conforme necessário
        df_update_inactive_new = df_joined.select([
            pl.col("nome"),
            pl.col("sobrenome"),
            pl.col("idade"),
            pl.col("email"),
            pl.col("data_cadastro"),
            pl.col("effective_date"),
            pl.col("effective_date").alias("end_date"),
            pl.lit(0).alias("is_current")
        ])
        df_insert = df_insert.join(
            df_update_inactive_new, 
            on="email", 
            how="left"
        ).select([
            pl.col("nome"),
            pl.col("sobrenome"),
            pl.col("idade"),
            pl.col("email"),
            pl.col("data_cadastro"),
            pl.when(pl.col("nome_right").is_not_null()).then(pl.col("effective_date_right")).otherwise(pl.col("effective_date")).alias("effective_date"),
            pl.when(pl.col("nome_right").is_not_null()).then(pl.col("end_date_right")).otherwise(pl.col("end_date")).alias("end_date"),
             pl.when(pl.col("nome_right").is_not_null()).then(pl.col("is_current_right")).otherwise(pl.col("is_current")).alias("is_current"),
        ])
        # Converta o DataFrame para um formato que pode ser enviado ao ClickHouse
        data = df_insert.to_dicts()
        
        # Inserir os dados no ClickHouse
        client_ch.execute(
            'INSERT INTO dim_clients (nome, sobrenome, idade, email, data_cadastro, effective_date, end_date, is_current) VALUES',
            [(row['nome'], row['sobrenome'], row['idade'], row['email'], row['data_cadastro'], row['effective_date'], row['end_date'], row['is_current']) for row in data]
        )
    

      

    

In [38]:
for i in range(40):
    content[i]['data_cadastro'] = '2025-12-12'

In [39]:
insert_dim_clients(content)

/tmp/ipykernel_423/3696115533.py:21: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df_table = pl.DataFrame(result, schema=columns)


In [9]:

from datetime import datetime
from kafka import KafkaProducer
import json
from faker import Faker
import random
from clickhouse_driver import Client
"""
Esta DAG apenas insere dados fakes de clientes no topico kafka
para exemplificar o processo.
"""


producer = KafkaProducer(bootstrap_servers='kafka:9092',
                value_serializer=lambda x: json.dumps(x).encode('utf-8'))

fake = Faker()
clientes = []
num_registros = 100  # Número total de registros


# Conexão ao servidor ClickHouse
client_ch = Client(
    host='clickhouse',       # Endereço do servidor ClickHouse
    port=9000,              # Porta padrão para conexões TCP
    user='default',         # Usuário (padrão é 'default')
    password='',            # Senha (por padrão, vazio)
    database='default',     # Nome do banco de dados (por padrão, 'default')
)
# Dicionário para armazenar dados de clientes já gerados

client_ch.execute('''
CREATE TABLE IF NOT EXISTS dim_clients (
    nome String,
    sobrenome String,
    idade Int32,
    email String,
    data_cadastro Date,
    effective_date Date,
    end_date Date,
    is_current UInt8
) ENGINE = MergeTree()
ORDER BY (nome, sobrenome, effective_date)
''')
clientes_existentes = client_ch.execute("SELECT nome, sobrenome, idade, email FROM dim_clients")
clientes_existentes_dict = {}
if len(clientes_existentes) > 0:
    clientes_existentes_dict = {cliente[3]: {"nome": cliente[0], "sobrenome": cliente[1], "idade": cliente[2]} for cliente in clientes_existentes}
for _ in range(num_registros):
    # Decidir se deve criar um novo cliente ou reutilizar um existente
    if clientes_existentes_dict and random.random() < 0.2:  # 20% de chance de reutilizar um cliente existente
        email = random.choice(list(clientes_existentes_dict.keys()))
        cliente_base = clientes_existentes_dict[email]
        cliente = {
            "nome": cliente_base["nome"],
            "sobrenome": cliente_base["sobrenome"],
            "idade": cliente_base["idade"],
            "email": email,
            "data_cadastro": fake.date_this_year().strftime("%Y-%m-%d")
        }
    else:
        email = fake.email()
        cliente = {
            "nome": fake.first_name(),
            "sobrenome": fake.last_name(),
            "idade": random.randint(18, 70),
            "email": email,
            "data_cadastro": fake.date_this_year().strftime("%Y-%m-%d")
        }
        clientes_existentes[email] = cliente

    clientes.append(cliente)





TypeError: list indices must be integers or slices, not str

In [10]:
clientes_existentes_dict

{}